<a href="https://colab.research.google.com/github/amanullahshah32/Deep-Learning/blob/main/PyTorch_Image_Segmentation_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF

In [9]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

class UNET(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature * 2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature * 2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1] * 2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx // 2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx + 1](concat_skip)

        return self.final_conv(x)



In [11]:
def test():
    x = torch.randn((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape

if __name__ == '__main__':
    test()


torch.Size([3, 1, 161, 161])
torch.Size([3, 1, 161, 161])


# **Import dataset**

In [18]:
!pip install kaggle

In [19]:
# Make a directory for the Kaggle API credentials
!mkdir -p ~/.kaggle

# Move kaggle.json to this directory
!cp kaggle.json ~/.kaggle/

# Set the appropriate permissions
!chmod 600 ~/.kaggle/kaggle.json


In [20]:
!kaggle competitions download -c carvana-image-masking-challenge


100% 24.4G/24.4G [04:57<00:00, 143MB/s]
100% 24.4G/24.4G [04:57<00:00, 88.0MB/s]


In [23]:
!unzip carvana-image-masking-challenge.zip -d carvana_data/


Archive:  carvana-image-masking-challenge.zip
  inflating: carvana_data/29bb3ece3180_11.jpg  
  inflating: carvana_data/metadata.csv.zip  
  inflating: carvana_data/sample_submission.csv.zip  
  inflating: carvana_data/test.zip   
  inflating: carvana_data/test_hq.zip  
  inflating: carvana_data/train.zip  
  inflating: carvana_data/train_hq.zip  
  inflating: carvana_data/train_masks.csv.zip  
  inflating: carvana_data/train_masks.zip  


In [ ]:
# Unzip the train and mask files
!unzip carvana_data/train.zip -d carvana_data/train
!unzip carvana_data/train_masks.zip -d carvana_data/train_masks


In [ ]:
import os
from PIL import Image
from torch.utils.data import dataset
import numpy as np

class CarvanaDataset(Dataset):
  def __init__(self, image_dir, mask_dir, transform=None):
    self.image_dir = image_dir
    self.mask_dir = mask_dir
    self.transform = transform
    self.images = os.listdir(image_dir)
  def __len__(self):
    return len(self.images)
  def __getitem__(self, index):
    img_path = os.path.join(self.image_dir, self.images[index])
    mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))
    iamge = np.array(Image.open(img_path).convert("RGB"))
    mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
    # 0.0 , 255
    mask[mask == 255.0] = 1.0
    if self.transform is not None:
      augmentations = self.transform(image=image, mask=mask)
      image = augmentations["image"]
      mask = augmentations["mask"]
    return image, mask

In [22]:
8

8

In [24]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision.transforms as transforms
import torch

# Define the CarvanaDataset class
class CarvanaDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)  # List of image file names

    def __len__(self):
        return len(self.images)  # Return the number of images

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", "_mask.gif"))

        # Load image and mask
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)

        # Normalize mask to binary values (0 and 1)
        mask[mask == 255.0] = 1.0

        if self.transform is not None:
            # Apply the transformations (augmentation) if provided
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask



In [25]:
# Example of applying basic PyTorch transformations
data_transforms = transforms.Compose([
    transforms.ToTensor(),  # Convert PIL Image or numpy array to tensor
    transforms.Resize((160, 240)),  # Resize to a consistent size (optional)
])

# Create the dataset object
train_dataset = CarvanaDataset(
    image_dir="carvana_data/train",   # Adjust the path according to your dataset
    mask_dir="carvana_data/train_masks",
    transform=data_transforms
)

# DataLoader to load data in batches
train_loader = DataLoader(
    train_dataset,
    batch_size=32,  # Load images in batches of 32
    shuffle=True
)

# Example of checking a batch
images, masks = next(iter(train_loader))
print(f"Images shape: {images.shape}")
print(f"Masks shape: {masks.shape}")


FileNotFoundError: [Errno 2] No such file or directory: 'carvana_data/train'